In [1]:
import os

os.chdir("..")

In [2]:
import torch
from diffusers import FluxKontextPipeline
from PIL import Image


/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`flash-attn` is not available or the version is too old. Please install `flash-attn>=2.6.3`.
`sageattention` is not available or the version is too old. Please install `sageattention>=2.1.1`.


In [3]:
# !export HF_HUB_CACHE=./cache

## Normal FluxKontext Generation

In [5]:
pipe = FluxKontextPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-Kontext-dev", torch_dtype=torch.bfloat16
)
pipe = pipe.to("cuda")


Loading pipeline components...: 100%|█████████████████████████████████████████| 7/7 [00:00<00:00,  7.14it/s]


## Extract the subject/character

In [6]:

def generate_image(base_scene_path, save_path, prompt = "Add character to the image.", width=None, height=None):
    base_scene = Image.open(base_scene_path).convert("RGB")

    if width is None:
        width, height = base_scene.size

    seed = torch.Generator().manual_seed(42)

    result_img_base = pipe(
        prompt=prompt,
        image=base_scene,
        num_inference_steps=28,
        height=height,
        width=width,
        # generator=seed,
        _auto_resize=False
    ).images[0]
    result_img_base.save(save_path)
    return result_img_base

100%|███████████████████████████████████████████████████████████████████████| 14/14 [00:17<00:00,  1.23s/it]


In [ ]:
src = "custom_data"

end_dir = os.path.join(src, "end")
image_extensions = (".png", ".jpg", ".jpeg", ".webp")
image_paths = [
    os.path.join(end_dir, fname)
    for fname in os.listdir(end_dir)
    if fname.lower().endswith(image_extensions)
]




In [ ]:
reference_prompt = "Extract the boy from the image, in a white background."
for image_path in image_paths:
    reference_save_path = image_path.replace("end", "reference")
    generate_image(image_path, reference_save_path, reference_prompt, 512, 512)

In [ ]:
start_prompt = "Remove the boy from the image."
for image_path in image_paths:
    start_save_path = image_path.replace("end", "start")
    generate_image(image_path, start_save_path, start_prompt)